
- Data Preprocess:
    1. change some abbreviated words into the normal form
    2. remove URL
    3. remove "<LH>" in the sentences
    4. lemmatize
    5.  & tokenization
- Feature Extraction by 1. BOW 2.TFIDF 
- Feature Extraction by selectKBest


In [10]:
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler,Dataset
#from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
#from keras import backend as K
#import keras.layers as layers
#from keras.models import Model, load_model
#from keras.engine import Layer
#import tensorflow_hub as hub
#import tensorflow as tf
from tqdm import tqdm
import pandas as pd
import joblib
import random
import numpy as np
import matplotlib.pyplot as plt
#from keras.preprocessing.sequence import pad_sequences
#from tensorflow.python.client import device_lib 
import multiprocessing
from sklearn.decomposition import PCA
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif

In [9]:
%run clean_tool.ipynb
!pip install import_ipynb
import import_ipynb
import data_tool 
import clean_tool as ct

  Created wheel for import-ipynb: filename=import_ipynb-0.1.3-cp37-none-any.whl size=2982 sha256=1a46ead5d354f0a8fb0488bba2765231cb7c235c0971dfefc598fb11323465e0
  Stored in directory: C:\Users\Lily\AppData\Local\pip\Cache\wheels\b4\7b\e9\a3a6e496115dffdb4e3085d0ae39ffe8a814eacc44bbf494b5
Successfully built import-ipynb
importing Jupyter notebook from data_tool.ipynb
importing Jupyter notebook from clean_tool.ipynb


In [16]:
# Import Data
X_data, Y_data, X_val = data_tool.loadData()
# Path or Direction  
savPath = './fea_sel/'

Loading Raw Data...
Loading training set...
Loading testing set...


#### Check the data 

In [17]:
X_data.head()

,_score,_index,_crawldate,_type,hashtags,tweet_id,text
0,391,hashtag_tweets,2015-05-23 11:42:47,tweets,[Snapchat],0x376b20,"People who post ""add me on #Snapchat"" must be ..."
1,433,hashtag_tweets,2016-01-28 04:52:09,tweets,"[freepress, TrumpLegacy, CNN]",0x2d5350,"@brianklaas As we see, Trump is dangerous to #..."
3,376,hashtag_tweets,2016-01-24 23:53:05,tweets,[],0x1cd5b0,Now ISSA is stalking Tasha 😂😂😂 <LH>
5,120,hashtag_tweets,2015-06-11 04:44:05,tweets,"[authentic, LaughOutLoud]",0x1d755c,@RISKshow @TheKevinAllison Thx for the BEST TI...
6,1021,hashtag_tweets,2015-08-18 02:30:07,tweets,[],0x2c91a8,Still waiting on those supplies Liscus. <LH>


In [18]:
Y_data.head()

,tweet_id,emotion
0,0x376b20,anticipation
1,0x2d5350,sadness
2,0x1cd5b0,fear
3,0x1d755c,joy
4,0x2c91a8,anticipation


In [19]:
X_val.head()

,_score,_index,_crawldate,_type,hashtags,tweet_id,text
2,232,hashtag_tweets,2017-12-25 04:39:20,tweets,[bibleverse],0x28b412,"Confident of your obedience, I write to you, k..."
4,989,hashtag_tweets,2016-01-08 17:18:59,tweets,[],0x2de201,"""Trust is not the same as faith. A friend is s..."
9,66,hashtag_tweets,2015-09-09 09:22:55,tweets,"[materialism, money, possessions]",0x218443,When do you have enough ? When are you satisfi...
30,104,hashtag_tweets,2015-10-10 14:33:26,tweets,"[GodsPlan, GodsWork]",0x2939d5,"God woke you up, now chase the day #GodsPlan #..."
33,310,hashtag_tweets,2016-10-23 08:49:50,tweets,[],0x26289a,"In these tough times, who do YOU turn to as yo..."


### Functions

In [12]:
def clean_data(data,col='text'):
    # data=text_tr
    # data1=text_tr['text'][idx]
    '''
    remove unneccessary URL,whitespace,punctuation
    lemmatization or stemming 
    tokenization
    !! Not Sure should or should not keep ',' '.'
    '''
    data['clean_sent'] = ct.FirstClean(data[col])
    data['clean_sent'] = ct.removeURL(data['clean_sent'])
    data['clean_sent'] = ct.removeWP(data['clean_sent'])
    data['clean_sent'] = ct.removeLH(data['clean_sent'])
    data['clean_sent'] = ct.lemma(data['clean_sent'])  
    data['clean_sent'] = ct.preprocess(data['clean_sent'])
    data['clean_sent'] = ct.removePUNC(data['clean_sent'])
    # data['clean_sent'] = ct.tokenizer(data['clean_sent'])
    return data

In [13]:
class Embedding():
    '''
    train: a list of text 
    '''
    def __init__(self,train,test = None):
        self.train=train
        self.test =test
        self.model = None
#    def word2vec(self):
#        from gensim.models import word2vec
#        from gensim import models
        
    def BOW(self):
        from sklearn.feature_extraction.text import CountVectorizer
        self.model = CountVectorizer(lowercase=True,max_features = 2048)
        # bow_tr,bow_ts = self.run()
        bow_tr = self.model.fit_transform(self.train)
        bow_ts = self.model.transform(self.test)
        return bow_tr,bow_ts
    
    def tfidf(self):
        from sklearn.feature_extraction.text import TfidfTransformer
        
        X_tr, X_ts  =self.BOW()
        self.model = TfidfTransformer()
        # tfidf_tr,tfidf_ts = self.run()
        tfidf_tr = self.model.fit_transform(X_tr)
        tfidf_ts = self.model.transform(X_ts)
        return tfidf_tr,tfidf_ts
    
    def run(self):
        self.model.fit(self.train)
        emb_tr = self.model.transform(self.train)
        # if self.test.empty == False :
        emb_ts = self.model.transform(self.test)
        return emb_tr,emb_ts
        return emb_tr,emb_tr

In [14]:
def label(Y_data):
    '''
    to transfer the emotion into 0-7 by orders
    '''
    emotion = ['sadness' ,'disgust' ,'anticipation', 'joy' ,'trust' ,'anger', 'fear','surprise']
    label = pd.DataFrame(np.zeros(len(Y_data)))
    for i,emo in enumerate(emotion):
        label.loc[Y_data['emotion']==emo]=i
    joblib.dump(label,'./fea_sel/label/label8.pkl')


In [ ]:
#if __name__ == '__main__':

In [20]:
pool = multiprocessing.Pool(processes=16)
col = ['_score','hashtags','tweet_id','text']
text_tr = X_data[col]
text_ts = X_val[col]

print('start')

start


lemmatization consumes lots of time, use muktiprocessing to speed up

In [21]:
# 1. Clean and lemmaitize  
data_tr = pool.map(clean_data(text_tr))
data_ts = pool.map(clean_data(text_ts))
#joblib.dump(data_tr, './fea_sel/text/data_train.pkl')
#joblib.dump(data_ts, './fea_sel/test/data_test.pkl')

sent_tr = data_tr['clean_sent']
sent_ts = data_ts['clean_sent']
# sent_tr=sent_tr.apply(lambda x: ' '.join(x))

cleaning...


KeyboardInterrupt: 

In [ ]:
## wordembedding
# 2-1. word2vec
# 2-2. tfidf
emb_tr,emb_ts = Embedding(sent_tr,sent_ts).tfidf()

# 2-3. BOW 
emb_tr,emb_ts  = Embedding(sent_tr,sent_ts).BOW()

# 2-4. glove

In [ ]:
# 3. other feature
hashtag_tr = data_tr['hashtags'].apply(lambda x: ' '.join(x))
hashtag_ts = data_ts['hashtags'].apply(lambda x: ' '.join(x))

# tmp_tr = pool.map(ct.lemma(hashtag_tr))
# tmp_ts = pool.map(ct.lemma(hashtag_ts))

emb_tr,emb_ts  = Embedding(hashtag_tr,hashtag_ts).BOW()

In [ ]:
# 4. feature selection, dimension reduction 
# pca = PCA(n_components=2)
y=joblib.load('./fea_sel/label/label8.pkl')
sel_f = SelectKBest(f_classif, k=512)
X_train_f = sel_f.fit_transform(emb_tr, y)
X_test_f = sel_f.transform(emb_ts)
joblib.dump(X_train_f,'./fea_sel/BOW/bow_train-512.pkl')

In [ ]:
# 5. concat. other feature and main feature
XX = joblib.load('./fea_sel/BOW/bow_test-2048.pkl')
xx = joblib.load('./fea_sel/hashtag_test_320.pkl')
import scipy.sparse as sp
con = sp.hstack((XX, xx))
con.shape
joblib.dump(con,'./fea_sel/BOW/bow_test-2368.pkl')